## Arkouda Registration Example
In this example we will:
- Create random pdarrays and Strings
- Register these objects
- Verify their registration status using `ak.list_registry`, `is_registered`, and `ak.info`
- Remove all non-register objects from the symbol table using `ak.clear`
- Disconnect from the arkouda server
- Reconnect to the arkouda server
- Attach to all registered objects
- Unregister everything
- Shutdown the server

Arkouda functions used:
- `ak.connect`
- `ak.randint`
- `ak.random_strings_uniform`
- `ak.pdarray.register`
- `ak.Strings.register`
- `ak.list_registry`
- `ak.pdarray.is_registered`
- `ak.Strings.is_registered`
- `ak.info`
- `ak.clear`
- `ak.disconnect`
- `ak.pdarray.attach`
- `ak.Strings.attach`
- `ak.pdarray.unregister`
- `ak.Strings.unregister`
- `ak.shutdown`


#### Launching and Connecting to Arkounda Server
Be sure to follow the installation instructions on the [Arkouda README](https://github.com/mhmerrill/arkouda#readme) before running this notebook

In [ ]:
import arkouda as ak

The arkouda server must be running before connecting with `ak.connect`. The server can be launched by navigating to the arkouda directory and running `./arkouda_server -nl 1`. For more info, refer to [Running arkouda_server](https://github.com/mhmerrill/arkouda#running-arkouda_server-toc)

In [ ]:
# connect to the arkouda server using the connect_url which the server prints out
ak.connect(connect_url="tcp://localhost:5555")

#### Intializing and Registering Variables
We create random `pdarray` and `Strings` objects and register them using in place `register` functions

In [ ]:
pda1 = ak.randint(0,10,100)
pda2 = ak.randint(0,10,100)
str1 = ak.random_strings_uniform(2, 5, 100)
str2 = ak.random_strings_uniform(2, 5, 100)

In [ ]:
pda1.register('pda1')
str1.register('str1')

#### Verifying Registration
We have just registered `pdarray pda1` and `String str1`. Note that `pdarray pda2` and `String str2` have not been registered. We can verify this using `ak.list_registry`, `is_registered`, or `ak.info`

In [ ]:
# ak.list_registry returns a python list of all registered object names
print(ak.list_registry())

In [ ]:
# Object.is_registered() returns a boolean indicating the objects registration status
print(f'pda1 is registered: {pda1.is_registered()}')
print(f'pda2 is registered: {pda2.is_registered()}')
print(f'str1 is registered: {str1.is_registered()}')
print(f'str2 is registered: {str2.is_registered()}')

In [ ]:
# ak.info returns all attributes of an object
# ak.info can be called with a single object, all registered objects, or all objects

print("ak.info('registered_object_name'):")
print(ak.info('pda1'))

print('ak.info(ak.RegisteredSymbols):')
print(ak.info(ak.RegisteredSymbols))

print('ak.info(ak.AllSymbols):')
print(ak.info(ak.AllSymbols))

We can see `ak.info(ak.RegisteredSymbols)` only contains references to objects related to `pda1` and `str1`

`ak.clear()` removes all non-registered objects from the symbol table, so `ak.info(ak.AllSymbols)` is different after a clear

In [ ]:
print('Before clear:')
print(ak.info(ak.AllSymbols))
ak.clear()
print('After clear:')
print(ak.info(ak.AllSymbols))


#### Disconnecting from Arkouda Server and Attaching Registered Objects
Users can `attach` to objects registered with the server after a disconnect. This enables access to registered objects even if the original python object is lost  

We are going to simulate the python client dying. To do this, we set references for `pda1` and `str1` to `None` and disconnect from the server

In [ ]:
pda1 = None
str1 = None

In [ ]:
ak.disconnect()

Now we come back and reconnect to the server. We attempt to access `pda1` and `str1` but no longer have the objects in python

In [ ]:
# connect to the arkouda server using the connect_url which the server prints out
ak.connect(connect_url="tcp://localhost:5555")

In [ ]:
print(f'pda1:{pda1}')
print(f'str1:{str1}')

However the server still has these objects registered

In [ ]:
print('After reconnect to server:')
print(ak.info(ak.AllSymbols))

We want to attach to these objects on the server to regain access to them

In [ ]:
pda1 = ak.pdarray.attach('pda1')
str1 = ak.Strings.attach('str1')

In [ ]:
print(f'pda1:{pda1}')
print(f'str1:{str1}')

We restored our access to `pda1` and `str1` using the `attach` functionality. Now we unregister everything and shutdown the arkouda server 

In [ ]:
pda1.unregister()
str1.unregister()
ak.clear()

In [ ]:
print(ak.info(ak.AllSymbols))

In [ ]:
ak.shutdown()